# Example of XANES convolution

- Author: Mauro Rovezzi <mauro.rovezzi@esrf.fr>
- Date: 2023-05

This notebook shows how to perform an energy-dependent convolution of XANES simulated data with the [FDMNES](https://fdmnes.neel.cnrs.fr/) code.

We load an example of experimental data from the [SSHADE/FAME database](https://www.sshade.eu/db/fame):

- [Zn K-edge XAS spectrum of ZnO](https://doi.org/10.26302/SSHADE/EXPERIMENT_ST_20180418_001)

In [1]:
import numpy as np
from larch.io import read_ascii
from larch.xafs import pre_edge

dat = read_ascii("../fdmnes/ZnO_SSHADE.data.txt", labels=("energy", "mu"))
pre_edge(dat)


A corresponding XAS simulation of Zn K-edge ZnO wurtzite is performed with FDMNES. We load both the non-convoluted data and the automatically convoluted one by FDMNES in order to compare with the convolution performed in Python here, that is, to show they are equivalent.

In [2]:
def read_fdmnes(path, labels=None):
    g = read_ascii(path, labels=labels)
    with open(path) as f:
        header = f.readline().split()
        g.e_edge = float(header[0])
        g.Z = header[1]
        g.e_fermi = float(header[6])
    return g

sim = read_fdmnes("../fdmnes/ZnO_FDMNES.txt", labels=("energy", "mu"))
sim_conv = read_ascii("../fdmnes/ZnO_FDMNES_conv.txt", labels=("energy", "mu"))


We do a convolution with a Lorentzian kernel. Need to create an array of energy dependent broadening parameters, starting from the XAS core-hole $\Gamma_{hole}$. *NOTE* all the parameters are described in the FDMNES manual.

In [3]:
from larch.math.convolution1D import atan_gamma, conv_fast

gamma_hole = 1.67 #from tabulated data

agammas = atan_gamma(sim.energy, e_cut=sim.e_fermi, gamma_hole=gamma_hole)

myconv_a = conv_fast(sim.energy, sim.mu, e_cut=sim.e_fermi, kernel="lorentzian", gammas=agammas)

To further simulate the experimental broadening, we may do an additional convolution with a Gaussian kernel

In [17]:
from larch.math.convolution1D import lin_gamma

lgammas = lin_gamma(sim.energy, gamma_hole=1)

myconv_b = conv_fast(sim.energy, myconv_a, e_cut=-10, kernel="gaussian", gammas=lgammas)


# make a few arrays for plotting:

fdmnes_conv_mu = sim_conv.mu / np.trapz(sim_conv.mu, sim_conv.energy)
fdmnes_conv_en = sim_conv.energy

larch_conv_mu = myconv_a / np.trapz(myconv_a, sim.energy)
larch_conv_en = sim.energy

exp_en = dat.energy - 9658
exp_mu = dat.norm * fdmnes_conv_mu[-10:].mean()


A simple plot to show that the FDMNES and Larch convoluted spectra are identical (**NOTE**: click on the plot legend to hide/show a curve)

In [20]:
# Plotting with wxmplot (as used by Larix): 
# Note: this will work when running on a Desktop, and show plots in a separate window, 
# but will work for notebooks on cloud services
from larch.wxlib import plot

plot(exp_en, exp_mu, label="exp data", show_legend=True, xmin=-15, xmax=70, new=True)
plot(fdmnes_conv_en, fdmnes_conv_mu, marker='+', label="FDMNES convolution")
plot(larch_conv_en, larch_conv_mu, label="Larch convolution")

In [22]:
# Plotting with Plotly
#TIP: uncomment the following two lines and restart the kernel if the Plotly widget does not show in your Jupyter notebook
import plotly.io as pio
pio.renderers.default = 'iframe'

from larch.plot.plotly_xafsplots import PlotlyFigure    
fig = PlotlyFigure()

fig.add_plot(exp_en, exp_mu, color="black", label="exp data")
fig.add_plot(fdmnes_conv_en, fdmnes_conv_mu, color="green", label="FDMNES convolution")
fig.add_plot(larch_conv_en, larch_conv_mu, color="red", label="Larch convolution")
#fig.add_plot(sim.energy, myconv_b / np.trapz(myconv_b, sim.energy), color="blue", label="Larch convolution + Gauss")
fig.set_xrange(-15, 70)

fig.set_style(width=900, height=500)
fig.show()

In [24]:
# Plotting with Bokeh: this seems more reliable with Jupyter than Plotly
from larch.plot.bokeh_xafsplots import BokehFigure    
fig = BokehFigure()

fig.add_plot(exp_en, exp_mu, color="black", label="exp data")
fig.add_plot(fdmnes_conv_en, fdmnes_conv_mu, color="green", label="FDMNES convolution")
fig.add_plot(larch_conv_en, larch_conv_mu, color="red", label="Larch convolution")
fig.set_xrange(-15, 70)
fig.show()


Loading BokehJS ...